In [2]:
import pandas as pd
import numpy as np

### EDA : 

In [3]:
df_train = pd.read_csv('train_F3WbcTw.csv')
df_test = pd.read_csv('test_tOlRoBf.csv')

In [4]:
print ("# of train datapoints = ", df_train.shape[0])
print ("# of test datapoints = ", df_test.shape[0])

# of train datapoints =  5279
# of test datapoints =  2924


### Finding Links in sentences : 

In [5]:

import re
temp_list = []
for i in df_train['text'].values:
    url = re.findall('http[s]?',i)
    if len(url)!=0:
        temp_list.append(url)
    
print ("Total links in sentences = ", len(temp_list))

    

Total links in sentences =  515


###### There are links in sentences so have to clean the sentences 

In [6]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [7]:

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [8]:
from tqdm import tqdm
from bs4 import BeautifulSoup
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(df_train['text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|█████████████████████████████████████| 5279/5279 [00:05<00:00, 988.23it/s]


In [9]:
df_train['cleaned_text'] = preprocessed_reviews

In [94]:
print (df_train['text'][11])
print (df_train['cleaned_text'][11])
print ("Drug name = ",df_train['drug'][11])

There is significant bleeding in the right eye, so that will need laser and possibly surgery longer term. There is thankfully less in the left, so they will treat with injections, then assess.  Click to expand... Ah, OK. It was just that you mentioned macular oedema (which doesn't cause vitreous haemorrhage) but didn't mention proliferative diabetic retinopathy or PDR (which can cause vitreous haemorrhages). I know you mentioned laser, but there are different types and sometimes macular oedema is treated with macular laser. The type used for PDR is called PRP (pan-retinal photocoagulation) and is applied to the periphery of the retina. There is some good information about injections for macular oedema here:-  http://www.diabeticretinopathy.org.uk/lucentispatientsdr.htm and good information about laser treatment here:-
significant bleeding right eye need laser possibly surgery longer term thankfully less left treat injections assess click expand ah ok mentioned macular oedema not cause 

In [96]:
drug_list = []
for drug_name in df_train['drug'].values :
    drug_name = re.sub('[^A-Za-z]+', ' ', drug_name)
    drug_name = ' '.join(e.lower() for e in drug_name.split() )
    drug_list.append(drug_name)

In [98]:
df_train['drug_cleaned'] = drug_list

In [14]:
df_train['sentiment'].value_counts() #imbalanced dataset

2    3825
1     837
0     617
Name: sentiment, dtype: int64

In [269]:
df_train[df_train['drug_cleaned']=='gilenya']['sentiment'].value_counts()

2    353
1    236
0     77
Name: sentiment, dtype: int64

In [15]:
df_train['drug'].value_counts()
# Many drugs occur more than one time

ocrevus                         676
gilenya                         666
ocrelizumab                     441
entyvio                         303
humira                          270
fingolimod                      238
remicade                        229
opdivo                          224
tarceva                         218
cladribine                      200
keytruda                        199
tagrisso                        161
stelara                         161
alimta                          146
lucentis                         67
eylea                            65
avastin                          52
nivolumab                        50
cimzia                           48
alectinib                        43
crizotinib                       43
vitrectomy                       41
simponi                          39
erlotinib                        35
tecentriq                        34
xalkori                          33
pemetrexed                       33
osimertinib                 

### Modelling : 

### Baseline  LSTM : 

In [206]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer()
reviews_vocab = count_vec.fit_transform(df_train['cleaned_text'].values)

In [207]:

word_count =  list(zip(count_vec.get_feature_names(),\
    np.asarray(reviews_vocab.sum(axis=0)).ravel()))

word_count_sorted = sorted(word_count , key=lambda x:x[1] , reverse=True)

In [18]:
print ("Taking top 5000 words from the vocab")
word_count_sorted = word_count_sorted[:15000]

Taking top 5000 words from the vocab


In [208]:
word_count_sorted = { j[0]:i  for i,j in enumerate(word_count_sorted,1) }
len(word_count_sorted)

35888

In [20]:
word_count_sorted.items()

dict_items([('not', 1), ('cancer', 2), ('patients', 3), ('treatment', 4), ('may', 5), ('ms', 6), ('disease', 7), ('also', 8), ('one', 9), ('would', 10), ('no', 11), ('new', 12), ('drug', 13), ('years', 14), ('lung', 15), ('therapy', 16), ('first', 17), ('time', 18), ('clinical', 19), ('like', 20), ('months', 21), ('get', 22), ('study', 23), ('people', 24), ('cells', 25), ('side', 26), ('doctor', 27), ('effects', 28), ('cell', 29), ('use', 30), ('trial', 31), ('drugs', 32), ('two', 33), ('weeks', 34), ('well', 35), ('blood', 36), ('could', 37), ('used', 38), ('know', 39), ('year', 40), ('see', 41), ('take', 42), ('risk', 43), ('good', 44), ('dose', 45), ('tumor', 46), ('surgery', 47), ('information', 48), ('health', 49), ('brain', 50), ('day', 51), ('symptoms', 52), ('crohn', 53), ('back', 54), ('much', 55), ('trials', 56), ('patient', 57), ('still', 58), ('pain', 59), ('small', 60), ('research', 61), ('multiple', 62), ('since', 63), ('results', 64), ('help', 65), ('every', 66), ('many'

In [21]:

tokenizer = count_vec.build_analyzer()


sentence_ids = []
for line in tqdm(df_train['cleaned_text'].values) :
    sentence = []
    for x in tokenizer(line):
        sentence.append( word_count_sorted.get(x,0) )
        
    sentence_ids.append(sentence)

100%|████████████████████████████████████| 5279/5279 [00:00<00:00, 6625.25it/s]


In [22]:
sentence_ids = np.array(sentence_ids)
sentence_ids.shape

(5279,)

In [23]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(df_train['sentiment'].values)
encoded_Y = encoder.transform(df_train['sentiment'].values)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.


In [25]:
from sklearn import model_selection

reviews_train, reviews_test, reviews_score_train , reviews_score_test = model_selection.train_test_split(sentence_ids, dummy_y, test_size=0.3, random_state=15)
print (len(reviews_train))
print (len(reviews_score_train))
print (len(reviews_test))
print (len(reviews_score_test))

3695
3695
1584
1584


In [26]:
from sklearn import model_selection
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Bidirectional

In [27]:
max_review_length = 500
X_train = sequence.pad_sequences(reviews_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(reviews_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(3695, 500)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0   

In [28]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [29]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(15000+1, embedding_vecor_length, input_length=max_review_length))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           480032    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 500, 200)          106400    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 603       
Total params: 827,835
Trainable params: 827,835
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
model.fit(X_train, reviews_score_train, nb_epoch=10, batch_size=64,validation_data=(X_test, reviews_score_test))
# Final evaluation of the model
scores = model.evaluate(X_test, reviews_score_test, verbose=0)
print("F1 Score: %.2f%%" % (scores[1]*100))

C:\Users\Amit.Peshwani\AppData\Local\Continuum\anaconda3\envs\doc-intel\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Instructions for updating:
Use tf.cast instead.
Train on 3695 samples, validate on 1584 samples
Epoch 1/10
3695/3695 [==============================] - ETA: 6:58 - loss: 1.1010 - f1: 0.0000e+ - ETA: 4:47 - loss: 1.0915 - f1: 0.0000e+ - ETA: 4:01 - loss: 1.0793 - f1: 0.0000e+ - ETA: 3:37 - loss: 1.0700 - f1: 0.0000e+ - ETA: 3:21 - loss: 1.0586 - f1: 0.0000e+ - ETA: 3:11 - loss: 1.0451 - f1: 0.0000e+ - ETA: 3:02 - loss: 1.0187 - f1: 0.1029   - ETA: 2:54 - loss: 0.9898 - f1: 0.18 - ETA: 2:47 - loss: 0.9663 - f1: 0.24 - ETA: 2:41 - loss: 0.9619 - f1: 0.28 - ETA: 2:36 - loss: 0.9553 - f1: 0.32 - ETA: 2:31 - loss: 0.9417 - f1: 0.35 - ETA: 2:26 - loss: 0.9271 - f1: 0.38 - ETA: 2:22 - loss: 0.9095 - f1: 0.41 - ETA: 2:18 - loss: 0.8982 - f1: 0.44 - ETA: 2:14 - loss: 0.8881 - f1: 0.46 - ETA: 2:10 - loss: 0.8804 - f1: 0.47 - ETA: 2:06 - loss: 0.8787 - f1: 0.48 - ETA: 2:02 - loss: 0.8762 - f1: 0.49 - ETA: 1:59 - loss: 0.8698 - f1: 0.51 - ETA: 1:58 - loss: 0.8635 - f1: 0.52 - ETA: 1:57 - loss: 0.86

3695/3695 [==============================] - ETA: 2:50 - loss: 0.2519 - f1: 0.92 - ETA: 2:50 - loss: 0.2136 - f1: 0.93 - ETA: 2:47 - loss: 0.2061 - f1: 0.92 - ETA: 2:43 - loss: 0.1799 - f1: 0.93 - ETA: 2:39 - loss: 0.1909 - f1: 0.93 - ETA: 2:35 - loss: 0.2061 - f1: 0.92 - ETA: 2:31 - loss: 0.2027 - f1: 0.92 - ETA: 2:28 - loss: 0.1976 - f1: 0.92 - ETA: 2:25 - loss: 0.2035 - f1: 0.92 - ETA: 2:22 - loss: 0.1913 - f1: 0.93 - ETA: 2:19 - loss: 0.1849 - f1: 0.93 - ETA: 2:16 - loss: 0.1874 - f1: 0.93 - ETA: 2:13 - loss: 0.1861 - f1: 0.93 - ETA: 2:10 - loss: 0.1805 - f1: 0.93 - ETA: 2:06 - loss: 0.1770 - f1: 0.93 - ETA: 2:03 - loss: 0.1774 - f1: 0.93 - ETA: 2:00 - loss: 0.1728 - f1: 0.93 - ETA: 1:57 - loss: 0.1753 - f1: 0.93 - ETA: 1:54 - loss: 0.1766 - f1: 0.93 - ETA: 1:51 - loss: 0.1794 - f1: 0.93 - ETA: 1:48 - loss: 0.1790 - f1: 0.93 - ETA: 1:45 - loss: 0.1765 - f1: 0.94 - ETA: 1:42 - loss: 0.1780 - f1: 0.93 - ETA: 1:39 - loss: 0.1786 - f1: 0.94 - ETA: 1:36 - loss: 0.1767 - f1: 0.94 - ETA: 

In [33]:
# Validation f1 score = 66 .(train f1 = 0.97)
# Try with less epochs as model started overfitting after epoch 2

### LSTM 2 : 

In [31]:
# Constant ID to the drugs 
# ID 1 to drugs name in both train and test data
# Check if f1 increases than the baseline LSTM model

In [209]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer()
reviews_vocab = count_vec.fit_transform(df_train['cleaned_text'].values)

In [210]:
word_count =  list(zip(count_vec.get_feature_names(),\
    np.asarray(reviews_vocab.sum(axis=0)).ravel()))

word_count_sorted = sorted(word_count , key=lambda x:x[1] , reverse=True)

In [211]:
word_count_sorted = { j[0]:i  for i,j in enumerate(word_count_sorted,2) }
len(word_count_sorted)

35888

In [212]:
drug_cleaned_seperated = []
for i in df_train['drug_cleaned'].unique() :
    temp1 = i.split(' ')
    for j in temp1 :
        drug_cleaned_seperated.append(j)

In [213]:
for i in drug_cleaned_seperated:
    word_count_sorted[i]=1

In [214]:
word_count_sorted['ranibizumab']

1

In [215]:
tokenizer = count_vec.build_analyzer()


sentence_ids = []
for line in tqdm(df_train['cleaned_text'].values) :
    sentence = []
    for x in tokenizer(line):
        sentence.append( word_count_sorted.get(x,0) )
        
    sentence_ids.append(sentence)

100%|████████████████████████████████████| 5279/5279 [00:00<00:00, 6360.24it/s]


In [216]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(df_train['sentiment'].values)
encoded_Y = encoder.transform(df_train['sentiment'].values)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [144]:
from sklearn import model_selection

reviews_train, reviews_test, reviews_score_train , reviews_score_test = model_selection.train_test_split(sentence_ids, dummy_y, test_size=0.3, random_state=15)
print (len(reviews_train))
print (len(reviews_score_train))
print (len(reviews_test))
print (len(reviews_score_test))

3695
3695
1584
1584


In [145]:
max_review_length = 500
X_train = sequence.pad_sequences(reviews_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(reviews_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(3695, 500)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0   

In [235]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(35888+2, embedding_vecor_length, input_length=max_review_length))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 500, 32)           1148480   
_________________________________________________________________
bidirectional_35 (Bidirectio (None, 500, 200)          106400    
_________________________________________________________________
bidirectional_36 (Bidirectio (None, 200)               240800    
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 603       
Total params: 1,496,283
Trainable params: 1,496,283
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train, reviews_score_train, nb_epoch=10, batch_size=64,validation_data=(X_test, reviews_score_test))
# Final evaluation of the model
scores = model.evaluate(X_test, reviews_score_test, verbose=0)
print("F1 Score: %.2f%%" % (scores[1]*100))

In [218]:
max_review_length = 500
X_train = sequence.pad_sequences(sentence_ids, maxlen=max_review_length)
# X_test = sequence.pad_sequences(reviews_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(5279, 500)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0   

In [183]:
estimator = model.fit(X_train, dummy_y, validation_split=0.20, epochs=1, batch_size=64)

Train on 4223 samples, validate on 1056 samples
Epoch 1/1
4223/4223 [==============================] - ETA: 10:50 - loss: 1.0989 - f1: 0.0000e+0 - ETA: 6:55 - loss: 1.0869 - f1: 0.0000e+0 - ETA: 5:36 - loss: 1.0734 - f1: 0.0000e+ - ETA: 4:54 - loss: 1.0597 - f1: 0.0000e+ - ETA: 4:28 - loss: 1.0482 - f1: 0.0000e+ - ETA: 4:10 - loss: 1.0238 - f1: 0.0000e+ - ETA: 3:56 - loss: 1.0055 - f1: 0.0937   - ETA: 3:46 - loss: 0.9772 - f1: 0.17 - ETA: 3:37 - loss: 0.9599 - f1: 0.23 - ETA: 3:29 - loss: 0.9475 - f1: 0.28 - ETA: 3:22 - loss: 0.9490 - f1: 0.32 - ETA: 3:15 - loss: 0.9346 - f1: 0.35 - ETA: 3:09 - loss: 0.9284 - f1: 0.38 - ETA: 3:03 - loss: 0.9196 - f1: 0.40 - ETA: 2:58 - loss: 0.9116 - f1: 0.42 - ETA: 2:53 - loss: 0.9020 - f1: 0.44 - ETA: 2:48 - loss: 0.8956 - f1: 0.46 - ETA: 2:44 - loss: 0.8907 - f1: 0.47 - ETA: 2:39 - loss: 0.8802 - f1: 0.49 - ETA: 2:35 - loss: 0.8794 - f1: 0.50 - ETA: 2:31 - loss: 0.8727 - f1: 0.51 - ETA: 2:27 - loss: 0.8700 - f1: 0.52 - ETA: 2:23 - loss: 0.8648 - f1:

In [187]:
estimator = model.fit(X_train, dummy_y, nb_epoch=5, batch_size=64)
# Final evaluation of the model
# scores = model.evaluate(X_test, reviews_score_test, verbose=0)
# print("F1 Score: %.2f%%" % (scores[1]*100))

C:\Users\Amit.Peshwani\AppData\Local\Continuum\anaconda3\envs\doc-intel\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/5
5279/5279 [==============================] - ETA: 15:10 - loss: 1.1008 - f1: 0.0000e+0 - ETA: 9:38 - loss: 1.0922 - f1: 0.0000e+0 - ETA: 7:42 - loss: 1.0803 - f1: 0.0000e+ - ETA: 6:47 - loss: 1.0666 - f1: 0.0000e+ - ETA: 6:09 - loss: 1.0573 - f1: 0.0000e+ - ETA: 5:42 - loss: 1.0482 - f1: 0.0000e+ - ETA: 5:25 - loss: 1.0246 - f1: 0.0000e+ - ETA: 5:12 - loss: 1.0060 - f1: 0.0777   - ETA: 5:00 - loss: 0.9920 - f1: 0.14 - ETA: 4:51 - loss: 0.9880 - f1: 0.19 - ETA: 4:46 - loss: 0.9859 - f1: 0.23 - ETA: 4:36 - loss: 0.9613 - f1: 0.27 - ETA: 4:27 - loss: 0.9447 - f1: 0.31 - ETA: 4:20 - loss: 0.9376 - f1: 0.34 - ETA: 4:12 - loss: 0.9282 - f1: 0.36 - ETA: 4:06 - loss: 0.9226 - f1: 0.38 - ETA: 4:00 - loss: 0.9193 - f1: 0.40 - ETA: 3:54 - loss: 0.9080 - f1: 0.42 - ETA: 3:48 - loss: 0.9036 - f1: 0.43 - ETA: 3:43 - loss: 0.8951 - f1: 0.45 - ETA: 3:38 - loss: 0.8932 - f1: 0.46 - ETA: 3:33 - loss: 0.8822 - f1: 0.47 - ETA: 3:28 - loss: 0.8713 - f1: 0.49 - ETA: 3:23 - loss: 0.8589 - f1: 0.50 

5279/5279 [==============================] - ETA: 4:02 - loss: 0.1863 - f1: 0.96 - ETA: 4:00 - loss: 0.2145 - f1: 0.94 - ETA: 3:57 - loss: 0.1707 - f1: 0.95 - ETA: 3:53 - loss: 0.1447 - f1: 0.96 - ETA: 3:50 - loss: 0.1365 - f1: 0.96 - ETA: 3:47 - loss: 0.1283 - f1: 0.96 - ETA: 3:44 - loss: 0.1383 - f1: 0.96 - ETA: 3:41 - loss: 0.1324 - f1: 0.96 - ETA: 3:39 - loss: 0.1255 - f1: 0.96 - ETA: 3:36 - loss: 0.1261 - f1: 0.96 - ETA: 3:33 - loss: 0.1332 - f1: 0.96 - ETA: 3:30 - loss: 0.1430 - f1: 0.95 - ETA: 3:27 - loss: 0.1469 - f1: 0.95 - ETA: 3:24 - loss: 0.1463 - f1: 0.95 - ETA: 3:20 - loss: 0.1505 - f1: 0.95 - ETA: 3:18 - loss: 0.1495 - f1: 0.95 - ETA: 3:15 - loss: 0.1447 - f1: 0.96 - ETA: 3:12 - loss: 0.1495 - f1: 0.95 - ETA: 3:09 - loss: 0.1489 - f1: 0.95 - ETA: 3:06 - loss: 0.1502 - f1: 0.95 - ETA: 3:03 - loss: 0.1453 - f1: 0.95 - ETA: 3:00 - loss: 0.1445 - f1: 0.95 - ETA: 2:57 - loss: 0.1442 - f1: 0.96 - ETA: 2:54 - loss: 0.1413 - f1: 0.96 - ETA: 2:51 - loss: 0.1468 - f1: 0.95 - ETA: 

In [188]:
model.save_weights('lstm_2_epoch_5')

### LSTM_2 with balanced classes : 

In [225]:
from sklearn.utils import class_weight
y = df_train['sentiment']
class_weights = class_weight.compute_class_weight('balanced',np.unique(y),y)

In [229]:
print (y.shape)

(5279,)


In [230]:
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')

Xtrain_sm,y_sm = smote.fit_sample(X_train,dummy_y)

In [231]:
print (y_sm.shape)

(8487, 3)


In [234]:
estimator = model.fit(Xtrain_sm , y_sm, validation_split=0.20, epochs=5, batch_size=64,class_weight=class_weights)

Train on 6789 samples, validate on 1698 samples
Epoch 1/5
6789/6789 [==============================] - ETA: 21:31 - loss: 1.0981 - f1: 0.0000e+0 - ETA: 13:26 - loss: 1.0954 - f1: 0.0000e+0 - ETA: 10:42 - loss: 1.0823 - f1: 0.0000e+0 - ETA: 9:17 - loss: 1.0753 - f1: 0.0000e+0 - ETA: 8:25 - loss: 1.0639 - f1: 0.0000e+ - ETA: 7:50 - loss: 1.0588 - f1: 0.0000e+ - ETA: 7:24 - loss: 1.0475 - f1: 0.0442   - ETA: 7:03 - loss: 1.0322 - f1: 0.11 - ETA: 6:46 - loss: 1.0383 - f1: 0.15 - ETA: 6:32 - loss: 1.0382 - f1: 0.19 - ETA: 6:20 - loss: 1.0354 - f1: 0.22 - ETA: 6:10 - loss: 1.0307 - f1: 0.24 - ETA: 6:01 - loss: 1.0266 - f1: 0.26 - ETA: 5:52 - loss: 1.0229 - f1: 0.24 - ETA: 5:44 - loss: 1.0169 - f1: 0.22 - ETA: 5:38 - loss: 1.0136 - f1: 0.21 - ETA: 5:31 - loss: 1.0111 - f1: 0.20 - ETA: 5:24 - loss: 1.0061 - f1: 0.18 - ETA: 5:18 - loss: 1.0066 - f1: 0.17 - ETA: 5:13 - loss: 1.0082 - f1: 0.17 - ETA: 5:07 - loss: 1.0057 - f1: 0.16 - ETA: 5:03 - loss: 1.0032 - f1: 0.18 - ETA: 4:58 - loss: 1.0010 -

6789/6789 [==============================] - ETA: 5:35 - loss: 0.5031 - f1: 0.81 - ETA: 5:31 - loss: 0.4169 - f1: 0.86 - ETA: 5:28 - loss: 0.4015 - f1: 0.86 - ETA: 5:25 - loss: 0.4152 - f1: 0.85 - ETA: 5:21 - loss: 0.4406 - f1: 0.84 - ETA: 5:17 - loss: 0.4166 - f1: 0.85 - ETA: 5:11 - loss: 0.4128 - f1: 0.85 - ETA: 5:06 - loss: 0.4232 - f1: 0.84 - ETA: 5:01 - loss: 0.4240 - f1: 0.84 - ETA: 4:56 - loss: 0.4192 - f1: 0.84 - ETA: 4:51 - loss: 0.4045 - f1: 0.85 - ETA: 4:48 - loss: 0.4182 - f1: 0.84 - ETA: 4:44 - loss: 0.4150 - f1: 0.84 - ETA: 4:40 - loss: 0.4285 - f1: 0.84 - ETA: 4:37 - loss: 0.4445 - f1: 0.83 - ETA: 4:34 - loss: 0.4467 - f1: 0.83 - ETA: 4:30 - loss: 0.4416 - f1: 0.83 - ETA: 4:27 - loss: 0.4324 - f1: 0.83 - ETA: 4:24 - loss: 0.4315 - f1: 0.83 - ETA: 4:22 - loss: 0.4329 - f1: 0.83 - ETA: 4:18 - loss: 0.4302 - f1: 0.83 - ETA: 4:15 - loss: 0.4300 - f1: 0.83 - ETA: 4:12 - loss: 0.4270 - f1: 0.84 - ETA: 4:09 - loss: 0.4310 - f1: 0.83 - ETA: 4:05 - loss: 0.4313 - f1: 0.83 - ETA: 

KeyboardInterrupt: 

In [236]:
model.fit(Xtrain_sm , y_sm, epochs=5, batch_size=64,class_weight=class_weights)

Epoch 1/5
8487/8487 [==============================] - ETA: 26:00 - loss: 1.0967 - f1: 0.0000e+0 - ETA: 15:47 - loss: 1.0887 - f1: 0.0000e+0 - ETA: 12:21 - loss: 1.0805 - f1: 0.0000e+0 - ETA: 10:37 - loss: 1.0741 - f1: 0.0000e+0 - ETA: 9:32 - loss: 1.0702 - f1: 0.0000e+0 - ETA: 8:48 - loss: 1.0614 - f1: 0.0000e+ - ETA: 8:16 - loss: 1.0458 - f1: 0.0000e+ - ETA: 7:51 - loss: 1.0273 - f1: 0.0000e+ - ETA: 7:32 - loss: 1.0053 - f1: 0.0000e+ - ETA: 7:21 - loss: 1.0040 - f1: 0.0328   - ETA: 7:07 - loss: 1.0118 - f1: 0.06 - ETA: 6:54 - loss: 1.0182 - f1: 0.10 - ETA: 6:43 - loss: 1.0248 - f1: 0.12 - ETA: 6:33 - loss: 1.0274 - f1: 0.11 - ETA: 6:25 - loss: 1.0198 - f1: 0.10 - ETA: 6:17 - loss: 1.0116 - f1: 0.09 - ETA: 6:14 - loss: 1.0097 - f1: 0.09 - ETA: 6:09 - loss: 1.0074 - f1: 0.08 - ETA: 6:03 - loss: 1.0098 - f1: 0.08 - ETA: 5:57 - loss: 1.0057 - f1: 0.07 - ETA: 5:51 - loss: 1.0019 - f1: 0.07 - ETA: 5:45 - loss: 1.0000 - f1: 0.07 - ETA: 5:40 - loss: 0.9991 - f1: 0.06 - ETA: 5:34 - loss: 0.99

8487/8487 [==============================] - ETA: 7:01 - loss: 0.4168 - f1: 0.86 - ETA: 7:53 - loss: 0.4089 - f1: 0.84 - ETA: 7:28 - loss: 0.4004 - f1: 0.85 - ETA: 7:16 - loss: 0.4082 - f1: 0.85 - ETA: 7:08 - loss: 0.4501 - f1: 0.83 - ETA: 7:09 - loss: 0.4480 - f1: 0.84 - ETA: 7:00 - loss: 0.4339 - f1: 0.84 - ETA: 7:00 - loss: 0.4319 - f1: 0.84 - ETA: 6:55 - loss: 0.4236 - f1: 0.84 - ETA: 6:48 - loss: 0.4156 - f1: 0.84 - ETA: 6:41 - loss: 0.3958 - f1: 0.85 - ETA: 6:35 - loss: 0.3910 - f1: 0.86 - ETA: 6:29 - loss: 0.3899 - f1: 0.85 - ETA: 6:24 - loss: 0.3848 - f1: 0.85 - ETA: 6:19 - loss: 0.3817 - f1: 0.86 - ETA: 6:14 - loss: 0.4019 - f1: 0.85 - ETA: 6:09 - loss: 0.3987 - f1: 0.85 - ETA: 6:05 - loss: 0.3957 - f1: 0.85 - ETA: 6:01 - loss: 0.3955 - f1: 0.85 - ETA: 6:00 - loss: 0.4064 - f1: 0.85 - ETA: 5:56 - loss: 0.4079 - f1: 0.85 - ETA: 5:52 - loss: 0.4067 - f1: 0.85 - ETA: 5:48 - loss: 0.4016 - f1: 0.85 - ETA: 5:44 - loss: 0.4060 - f1: 0.85 - ETA: 5:40 - loss: 0.4072 - f1: 0.85 - ETA: 

8487/8487 [==============================] - ETA: 6:34 - loss: 0.1340 - f1: 0.96 - ETA: 6:30 - loss: 0.1047 - f1: 0.98 - ETA: 6:37 - loss: 0.1051 - f1: 0.98 - ETA: 9:39 - loss: 0.1156 - f1: 0.98 - ETA: 9:01 - loss: 0.1457 - f1: 0.97 - ETA: 8:31 - loss: 0.1432 - f1: 0.96 - ETA: 8:09 - loss: 0.1556 - f1: 0.96 - ETA: 7:51 - loss: 0.1595 - f1: 0.96 - ETA: 7:36 - loss: 0.1555 - f1: 0.96 - ETA: 7:24 - loss: 0.1609 - f1: 0.95 - ETA: 7:13 - loss: 0.1684 - f1: 0.95 - ETA: 7:04 - loss: 0.1580 - f1: 0.95 - ETA: 6:56 - loss: 0.1640 - f1: 0.95 - ETA: 6:48 - loss: 0.1631 - f1: 0.95 - ETA: 6:41 - loss: 0.1686 - f1: 0.95 - ETA: 6:35 - loss: 0.1649 - f1: 0.95 - ETA: 6:29 - loss: 0.1624 - f1: 0.95 - ETA: 6:23 - loss: 0.1595 - f1: 0.95 - ETA: 6:17 - loss: 0.1601 - f1: 0.95 - ETA: 6:12 - loss: 0.1597 - f1: 0.95 - ETA: 6:07 - loss: 0.1567 - f1: 0.95 - ETA: 6:02 - loss: 0.1547 - f1: 0.95 - ETA: 5:58 - loss: 0.1584 - f1: 0.95 - ETA: 5:53 - loss: 0.1614 - f1: 0.95 - ETA: 5:49 - loss: 0.1589 - f1: 0.95 - ETA: 

In [237]:
model.save_weights('lstm_2_epoch_5_SMOTE_Balanced')

### Converting to cateogarical :

In [ ]:
# https://stackoverflow.com/questions/51360827/how-to-combine-numerical-and-categorical-values-in-a-vector-as-input-for-lstm

In [256]:
df_train['drug'].dtype

dtype('O')

In [257]:
df_train['drug'].unique().shape

(102,)

In [264]:
drug_unique = np.concatenate((df_train['drug'].unique(),df_test['drug'].unique()))
drug_unique = set(drug_unique)

111

In [268]:
drug_unique = list(drug_unique)

In [269]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(drug_unique)
encoded_Y = encoder.transform(df_train['drug'].values)
# convert integers to dummy variables (i.e. one hot encoded)
drug_cateogary = np_utils.to_categorical(encoded_Y)

In [271]:
drug_cateogary[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [289]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer()
reviews_vocab = count_vec.fit_transform(df_train['cleaned_text'].values)

In [290]:

word_count =  list(zip(count_vec.get_feature_names(),\
    np.asarray(reviews_vocab.sum(axis=0)).ravel()))

word_count_sorted = sorted(word_count , key=lambda x:x[1] , reverse=True)

In [291]:
word_count_sorted = { j[0]:i  for i,j in enumerate(word_count_sorted,2) }
len(word_count_sorted)

35888

In [292]:
tokenizer = count_vec.build_analyzer()


sentence_ids = []
for line in tqdm(df_train['cleaned_text'].values) :
    sentence = []
    for x in tokenizer(line):
        sentence.append( word_count_sorted.get(x,0) )
        
    sentence_ids.append(sentence)

100%|████████████████████████████████████| 5279/5279 [00:00<00:00, 5768.82it/s]


In [293]:
sentence_ids = np.array(sentence_ids)
sentence_ids.shape

(5279,)

In [294]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(df_train['sentiment'].values)
encoded_Y = encoder.transform(df_train['sentiment'].values)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [295]:
max_review_length = 500
X_train = sequence.pad_sequences(sentence_ids, maxlen=max_review_length)
# X_test = sequence.pad_sequences(reviews_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(5279, 500)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0   

In [299]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(35888+2, embedding_vecor_length, input_length=611))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 611, 32)           1148480   
_________________________________________________________________
bidirectional_43 (Bidirectio (None, 611, 200)          106400    
_________________________________________________________________
bidirectional_44 (Bidirectio (None, 200)               240800    
_________________________________________________________________
dense_22 (Dense)             (None, 3)                 603       
Total params: 1,496,283
Trainable params: 1,496,283
Non-trainable params: 0
_________________________________________________________________
None


In [281]:
X_train.shape

(5279, 500)

In [282]:
drug_cateogary.shape

(5279, 111)

In [297]:
X = np.concatenate((X_train,drug_cateogary),axis=1)

In [298]:
estimator = model.fit(X, dummy_y, validation_split=0.20, epochs=1, batch_size=64)

Train on 4223 samples, validate on 1056 samples
Epoch 1/1
4223/4223 [==============================] - ETA: 15:45 - loss: 1.0985 - f1: 0.0000e+0 - ETA: 9:56 - loss: 1.0829 - f1: 0.0000e+0 - ETA: 7:54 - loss: 1.0641 - f1: 0.0000e+ - ETA: 6:51 - loss: 1.0470 - f1: 0.0000e+ - ETA: 6:11 - loss: 1.0303 - f1: 0.0000e+ - ETA: 5:43 - loss: 1.0008 - f1: 0.1212   - ETA: 5:22 - loss: 0.9803 - f1: 0.20 - ETA: 5:06 - loss: 0.9365 - f1: 0.27 - ETA: 4:53 - loss: 0.9597 - f1: 0.32 - ETA: 4:41 - loss: 0.9727 - f1: 0.35 - ETA: 4:30 - loss: 0.9503 - f1: 0.39 - ETA: 4:21 - loss: 0.9292 - f1: 0.42 - ETA: 4:13 - loss: 0.9011 - f1: 0.45 - ETA: 4:05 - loss: 0.8945 - f1: 0.47 - ETA: 3:57 - loss: 0.8902 - f1: 0.49 - ETA: 3:50 - loss: 0.8842 - f1: 0.50 - ETA: 3:44 - loss: 0.8788 - f1: 0.52 - ETA: 3:37 - loss: 0.8767 - f1: 0.53 - ETA: 3:31 - loss: 0.8757 - f1: 0.53 - ETA: 3:25 - loss: 0.8755 - f1: 0.54 - ETA: 3:20 - loss: 0.8760 - f1: 0.55 - ETA: 3:14 - loss: 0.8739 - f1: 0.55 - ETA: 3:09 - loss: 0.8686 - f1: 0.5

In [303]:
estimator = model.fit(X, dummy_y,  epochs=3, batch_size=64, class_weight=class_weights)

Epoch 1/3
5279/5279 [==============================] - ETA: 12:58 - loss: 1.0991 - f1: 0.0000e+0 - ETA: 9:22 - loss: 1.0860 - f1: 0.0000e+0 - ETA: 8:06 - loss: 1.0733 - f1: 0.0000e+ - ETA: 7:26 - loss: 1.0528 - f1: 0.0000e+ - ETA: 7:01 - loss: 1.0340 - f1: 0.0000e+ - ETA: 6:43 - loss: 0.9936 - f1: 0.1349   - ETA: 6:28 - loss: 0.9594 - f1: 0.22 - ETA: 6:13 - loss: 0.9496 - f1: 0.28 - ETA: 5:59 - loss: 0.9337 - f1: 0.33 - ETA: 5:47 - loss: 0.8940 - f1: 0.38 - ETA: 5:37 - loss: 0.8960 - f1: 0.41 - ETA: 5:28 - loss: 0.9007 - f1: 0.43 - ETA: 5:20 - loss: 0.8858 - f1: 0.45 - ETA: 5:12 - loss: 0.8841 - f1: 0.47 - ETA: 5:05 - loss: 0.8775 - f1: 0.48 - ETA: 4:58 - loss: 0.8697 - f1: 0.50 - ETA: 4:51 - loss: 0.8590 - f1: 0.52 - ETA: 4:48 - loss: 0.8599 - f1: 0.53 - ETA: 4:49 - loss: 0.8625 - f1: 0.53 - ETA: 4:43 - loss: 0.8584 - f1: 0.54 - ETA: 4:37 - loss: 0.8545 - f1: 0.55 - ETA: 4:30 - loss: 0.8541 - f1: 0.56 - ETA: 4:24 - loss: 0.8486 - f1: 0.57 - ETA: 4:19 - loss: 0.8443 - f1: 0.58 - ETA: 4

In [304]:
model.save_weights('bilstm_epoc_3_cateogarical')

#### Test Lstm : 

In [238]:
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(df_test['text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|█████████████████████████████████████| 2924/2924 [00:03<00:00, 890.76it/s]


In [239]:
df_test['cleaned_text']=preprocessed_reviews

In [241]:
for i in df_test['drug'].unique() :
    word_count_sorted[i]=1

In [242]:
sentence_ids = []
for line in tqdm(df_test['cleaned_text'].values) :
    sentence = []
    for x in tokenizer(line):
        sentence.append( word_count_sorted.get(x,0) )
        
    sentence_ids.append(sentence)

100%|████████████████████████████████████| 2924/2924 [00:00<00:00, 5067.59it/s]


In [243]:
sentence_ids = np.array(sentence_ids)
sentence_ids.shape

(2924,)

In [244]:
max_review_length = 500
X_test = sequence.pad_sequences(sentence_ids, maxlen=max_review_length)

print(X_test.shape)
print(X_test[1])

(2924, 500)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0

In [245]:
predictions = model.predict(X_test)

In [246]:
prediction = []
for i in predictions :
    prediction.append(np.argmax(i))

In [247]:
df = pd.DataFrame()
df['unique_hash'] = df_test['unique_hash'].values
df['sentiment']= prediction

In [248]:
df.head()

,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,0
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,0
2,50b6d851bcff4f35afe354937949e9948975adf7,2
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,2
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,2


In [249]:
df['sentiment'].value_counts()

2    2349
0     334
1     241
Name: sentiment, dtype: int64

In [204]:
df['sentiment'].value_counts()

2    2502
1     230
0     192
Name: sentiment, dtype: int64

In [250]:
df.to_csv('lstm_2_epoch_5_smote_balanced.csv',index=False)

### Test BiLstm : 

In [305]:
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(df_test['text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|█████████████████████████████████████| 2924/2924 [00:03<00:00, 843.28it/s]


In [306]:
df_test['cleaned_text']=preprocessed_reviews

In [ ]:
for i in df_test['drug'].unique() :
    word_count_sorted[i]=1

In [308]:

encoder = LabelEncoder()
encoder.fit(drug_unique)
encoded_Y = encoder.transform(df_test['drug'].values)
# convert integers to dummy variables (i.e. one hot encoded)
drug_cateogary_test = np_utils.to_categorical(encoded_Y)

In [309]:
sentence_ids = []
for line in tqdm(df_test['cleaned_text'].values) :
    sentence = []
    for x in tokenizer(line):
        sentence.append( word_count_sorted.get(x,0) )
        
    sentence_ids.append(sentence)
    
sentence_ids = np.array(sentence_ids)
sentence_ids.shape


100%|████████████████████████████████████| 2924/2924 [00:00<00:00, 5364.89it/s]


(2924,)

In [310]:
max_review_length = 500
X_test = sequence.pad_sequences(sentence_ids, maxlen=max_review_length)

print(X_test.shape)
print(X_test[1])

(2924, 500)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0

In [311]:
X = np.concatenate((X_test,drug_cateogary_test),axis=1)

In [312]:
X.shape

(2924, 611)

In [314]:
predictions = model.predict(X)

In [315]:
prediction = []
for i in predictions :
    prediction.append(np.argmax(i))

In [316]:
df = pd.DataFrame()
df['unique_hash'] = df_test['unique_hash'].values
df['sentiment']= prediction

In [317]:
df.head()

,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,2
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,2
2,50b6d851bcff4f35afe354937949e9948975adf7,2
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,2
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,2


In [319]:
df['sentiment'].value_counts()

2    2924
Name: sentiment, dtype: int64

### Catboost : 

In [81]:
df_train['cleaned_text'].shape

(5279,)

In [82]:
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(df_test['text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|█████████████████████████████████████| 2924/2924 [00:03<00:00, 916.96it/s]


In [83]:
df_test['cleaned_text']=preprocessed_reviews

In [84]:
drug_list = df_train['drug'].unique()
drug_lookup = { j:i  for i,j in enumerate(drug_list,1) }

In [85]:
drug_cat = []
for i in df_train['drug'].values :
    drug_cat.append(drug_lookup[i])

In [86]:
df_train['drug_cat'] = drug_cat

In [87]:
from sklearn import model_selection

reviews_train, reviews_test, reviews_score_train , reviews_score_test = model_selection.train_test_split(df_train,df_train['sentiment'] , test_size=0.3, random_state=15)
print (len(reviews_train))
print (len(reviews_score_train))
print (len(reviews_test))
print (len(reviews_score_test))

3695
3695
1584
1584


In [88]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() #in scikit-learn
reviews_train_bow = count_vect.fit_transform(reviews_train['cleaned_text'].values)
# reviews_cv_bow = count_vect.transform(reviews_cv['CleanedText'].values)
reviews_test_bow = count_vect.transform(reviews_test['cleaned_text'].values)

print("the type of count vectorizer ",type(reviews_train_bow))
print (reviews_train_bow.shape)
# print (reviews_cv_bow.shape)
print (reviews_test_bow.shape)

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
(3695, 30191)
(1584, 30191)


In [23]:
reviews_train_bow = np.array(reviews_train_bow.todense())
reviews_test_bow = np.array(reviews_test_bow.todense())

In [102]:
from gensim.models import Word2Vec

i=0
list_of_sent =[]
for sent in reviews_train['cleaned_text'].values:
    list_of_sent.append(sent.split())
    
w2v_model=Word2Vec(list_of_sent,min_count=5,size=300, workers=4)

w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

sent_vectors = [] #List to store all the avg w2vec's 

for sent in tqdm(reviews_train['cleaned_text']):
    cnt = 0 #to count no of words in each reviews
    sent_vec = np.zeros(300) #Initializing with zeroes
#     print("sent:",sent) 
#     sent = sent.decode("utf-8") 
    for word in sent.split():
        try:
#             print(word)
            wvec = w2v_model.wv[word] #Vector of each using w2v model
#             print("wvec:",wvec)
            sent_vec += wvec #Adding the vectors
#             print("sent_vec:",sent_vec)
            cnt += 1
        except: 
            pass #When the word is not in the dictionary then do nothing  
#     print(sent_vec)
    sent_vec /= cnt 
    sent_vectors.append(sent_vec) 
sent_vectors_train = np.array(sent_vectors)
# savefile(sent_vectors , 'AvgW2V_RF_Train')

number of words that occured minimum 5 times  10414
sample words  ['used', 'treat', 'skin', 'cancer', 'lung', 'need', 'tell', 'doctor', 'take', 'mekinist', 'allergy', 'trametinib', 'part', 'allergic', 'drugs', 'like', 'one', 'foods', 'substances', 'signs', 'rash', 'hives', 'itching', 'shortness', 'breath', 'wheezing', 'cough', 'swelling', 'face', 'lips', 'tongue', 'throat', 'breast', 'feeding', 'not', 'feed', 'months', 'last', 'dose', 'medicine', 'may', 'interact', 'health', 'problems', 'pharmacist', 'prescription', 'otc', 'natural', 'products', 'vitamins']


100%|█████████████████████████████████████| 3695/3695 [00:05<00:00, 718.65it/s]


In [103]:
from gensim.models import Word2Vec

# i=0
# list_of_sent =[]
# for sent in reviews_test['cleaned_text'].values:
#     list_of_sent.append(sent.split())
    
# w2v_model=Word2Vec(list_of_sent,min_count=5,size=300, workers=4)

w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

sent_vectors = [] #List to store all the avg w2vec's 

for sent in tqdm(reviews_test['cleaned_text']):
    cnt = 0 #to count no of words in each reviews
    sent_vec = np.zeros(300) #Initializing with zeroes
#     print("sent:",sent) 
#     sent = sent.decode("utf-8") 
    for word in sent.split():
        try:
#             print(word)
            wvec = w2v_model.wv[word] #Vector of each using w2v model
#             print("wvec:",wvec)
            sent_vec += wvec #Adding the vectors
#             print("sent_vec:",sent_vec)
            cnt += 1
        except: 
            pass #When the word is not in the dictionary then do nothing  
#     print(sent_vec)
    sent_vec /= cnt 
    sent_vectors.append(sent_vec) 
sent_vectors_test = np.array(sent_vectors)
# savefile(sent_vectors , 'AvgW2V_RF_Train')

number of words that occured minimum 5 times  10414
sample words  ['used', 'treat', 'skin', 'cancer', 'lung', 'need', 'tell', 'doctor', 'take', 'mekinist', 'allergy', 'trametinib', 'part', 'allergic', 'drugs', 'like', 'one', 'foods', 'substances', 'signs', 'rash', 'hives', 'itching', 'shortness', 'breath', 'wheezing', 'cough', 'swelling', 'face', 'lips', 'tongue', 'throat', 'breast', 'feeding', 'not', 'feed', 'months', 'last', 'dose', 'medicine', 'may', 'interact', 'health', 'problems', 'pharmacist', 'prescription', 'otc', 'natural', 'products', 'vitamins']


100%|█████████████████████████████████████| 1584/1584 [00:02<00:00, 700.77it/s]


In [104]:
reviews_train_bow = np.column_stack((sent_vectors_train,reviews_train['drug_cat'].values))
reviews_test_bow = np.column_stack((sent_vectors_test,reviews_test['drug_cat'].values))

In [119]:
type(reviews_train_bow[0][300])

numpy.float64

In [21]:
reviews_train_bow.shape

(3695, 30191)

In [27]:
reviews_train['drug'].values[:,None]

array([['dabrafenib'],
       ['ocrevus'],
       ['humira'],
       ...,
       ['tarceva'],
       ['ocrevus'],
       ['ocrelizumab']], dtype=object)

In [89]:
from scipy.sparse import hstack

reviews_train_bow = hstack((reviews_train_bow,reviews_train['drug_cat'].values[:,None]))
reviews_test_bow = hstack((reviews_test_bow,reviews_test['drug_cat'].values[:,None]))

In [57]:
import catboost

In [ ]:

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score

m=CatBoostClassifier(n_estimators=3000,random_state=15,eval_metric='F1',max_depth=6,learning_rate=0.001,od_wait=50
                     ,l2_leaf_reg=5,cat_features=[300],
                     use_best_model=True)
m.fit(reviews_train_bow,reviews_score_train,eval_set=[(reviews_train_bow,reviews_score_train),(reviews_test_bow,reviews_score_test)], early_stopping_rounds=100,verbose=100)

In [78]:

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score

m=CatBoostClassifier(n_estimators=3000,random_state=15,custom_metric='F1',max_depth=6,learning_rate=0.001,od_wait=50
                     ,l2_leaf_reg=5,cat_features=[300],
                     use_best_model=True)
m.fit(reviews_train_bow,reviews_score_train,eval_set=[(reviews_train_bow,reviews_score_train),(reviews_test_bow,reviews_score_test)], early_stopping_rounds=100,verbose=100)

0:	learn: 1.0979227	test: 1.0979227	test1: 1.0979456	best: 1.0979456 (0)	total: 1.45s	remaining: 1h 12m 14s


KeyboardInterrupt: 

In [ ]:
cb = catboost.CatBoostClassifier(n_estimators=4000, # use large n_estimators deliberately to make use of the early stopping
                         loss_function='Logloss',
                         eval_metric='AUC',
                         boosting_type='Ordered', # use permutations
                         random_seed=1994, 
                         use_best_model=True)
cb_optimizer = CatboostOptimizer(cb, X_train, y_train,categorical_columns_indices=categorical_features_indices)
m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val, y_val)], early_stopping_rounds=100,verbose=100)
p2=m.predict_proba(X_val)[:,-1]

### Lightgbm  :

In [108]:
reviews_train_bow.shape

(3695, 301)

In [95]:
# import lightgbm
from sklearn.metrics import f1_score
def evaluate_macroF1_lgb(truth, predictions):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [123]:
import lightgbm as lgb
# fit_params={"early_stopping_rounds":300, 
#             "eval_metric" : evaluate_macroF1_lgb, 
#             "eval_set" : [(reviews_test_bow,reviews_score_test)],
#             'eval_names': ['valid'],
#             #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
#             'verbose': True,
#             'categorical_feature': }
fit_params={"early_stopping_rounds":300, 
            "eval_metric" : evaluate_macroF1_lgb, 
            "eval_set" : [(reviews_test_bow,reviews_score_test)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': True }

In [128]:
clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.05, objective='multiclass',
                         random_state=15, silent=True, metric='None', 
                         n_jobs=4, n_estimators=5000, is_unbalance=True)

In [129]:
clf.fit(reviews_train_bow,reviews_score_train, **fit_params, categorical_feature=[300])

C:\Users\Amit.Peshwani\AppData\Local\Continuum\anaconda3\envs\doc-intel\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [300]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Amit.Peshwani\AppData\Local\Continuum\anaconda3\envs\doc-intel\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[1]	valid's macroF1: 0.278859
Training until validation scores don't improve for 300 rounds.
[2]	valid's macroF1: 0.278859
[3]	valid's macroF1: 0.278859
[4]	valid's macroF1: 0.278859
[5]	valid's macroF1: 0.278859
[6]	valid's macroF1: 0.278859
[7]	valid's macroF1: 0.278859
[8]	valid's macroF1: 0.278859
[9]	valid's macroF1: 0.278859
[10]	valid's macroF1: 0.278859
[11]	valid's macroF1: 0.278859
[12]	valid's macroF1: 0.278859
[13]	valid's macroF1: 0.278859
[14]	valid's macroF1: 0.278859
[15]	valid's macroF1: 0.278859
[16]	valid's macroF1: 0.281433
[17]	valid's macroF1: 0.281433
[18]	valid's macroF1: 0.281281
[19]	valid's macroF1: 0.284049
[20]	valid's macroF1: 0.286706
[21]	valid's macroF1: 0.291818
[22]	valid's macroF1: 0.299339
[23]	valid's macroF1: 0.302137
[24]	valid's macroF1: 0.309353
[25]	valid's macroF1: 0.309353
[26]	valid's macroF1: 0.311799
[27]	valid's macroF1: 0.313939
[28]	valid's macroF1: 0.317593
[29]	valid's macroF1: 0.319356
[30]	valid's macroF1: 0.324246
[31]	valid's mac

[259]	valid's macroF1: 0.398897
[260]	valid's macroF1: 0.398897
[261]	valid's macroF1: 0.396318
[262]	valid's macroF1: 0.394482
[263]	valid's macroF1: 0.394079
[264]	valid's macroF1: 0.396727
[265]	valid's macroF1: 0.396727
[266]	valid's macroF1: 0.396727
[267]	valid's macroF1: 0.394334
[268]	valid's macroF1: 0.396166
[269]	valid's macroF1: 0.39588
[270]	valid's macroF1: 0.396033
[271]	valid's macroF1: 0.396166
[272]	valid's macroF1: 0.394334
[273]	valid's macroF1: 0.394334
[274]	valid's macroF1: 0.396166
[275]	valid's macroF1: 0.393153
[276]	valid's macroF1: 0.391725
[277]	valid's macroF1: 0.394334
[278]	valid's macroF1: 0.393561
[279]	valid's macroF1: 0.393153
[280]	valid's macroF1: 0.393153
[281]	valid's macroF1: 0.391322
[282]	valid's macroF1: 0.396166
[283]	valid's macroF1: 0.396318
[284]	valid's macroF1: 0.396318
[285]	valid's macroF1: 0.398144
[286]	valid's macroF1: 0.397988
[287]	valid's macroF1: 0.397988
[288]	valid's macroF1: 0.397988
[289]	valid's macroF1: 0.398144
[290]	val

[516]	valid's macroF1: 0.388644
[517]	valid's macroF1: 0.388644
[518]	valid's macroF1: 0.389032
[519]	valid's macroF1: 0.389032
[520]	valid's macroF1: 0.389032
[521]	valid's macroF1: 0.389032
[522]	valid's macroF1: 0.389032
[523]	valid's macroF1: 0.390502
[524]	valid's macroF1: 0.39235
[525]	valid's macroF1: 0.392637
[526]	valid's macroF1: 0.39235
[527]	valid's macroF1: 0.39524
[528]	valid's macroF1: 0.393787
[529]	valid's macroF1: 0.393787
[530]	valid's macroF1: 0.396774
[531]	valid's macroF1: 0.39494
[532]	valid's macroF1: 0.396476
[533]	valid's macroF1: 0.393502
[534]	valid's macroF1: 0.393502
[535]	valid's macroF1: 0.393502
[536]	valid's macroF1: 0.393502
[537]	valid's macroF1: 0.395038
[538]	valid's macroF1: 0.394642
[539]	valid's macroF1: 0.394642
[540]	valid's macroF1: 0.394642
[541]	valid's macroF1: 0.394642
[542]	valid's macroF1: 0.395336
[543]	valid's macroF1: 0.395038
[544]	valid's macroF1: 0.394642
[545]	valid's macroF1: 0.394642
[546]	valid's macroF1: 0.394642
[547]	valid'

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', is_unbalance=True, learning_rate=0.05,
               max_depth=-1, metric='None', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=5000,
               n_jobs=4, num_leaves=31, objective='multiclass', random_state=15,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

### Lightgbm on full data : 

In [130]:
df_train.head()

,unique_hash,text,drug,sentiment,cleaned_text,drug_cat
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2,autoimmune diseases tend come clusters gilenya...,1
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2,completely understand would want try results r...,1
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2,interesting targets receptors rather like fing...,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2,interesting grand merci wonder lemtrada ocrevu...,3
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1,hi everybody latest mri results brain cervical...,1


In [132]:
drug_cat = []
for i in df_test['drug'].values :
    drug_cat.append(drug_lookup.get(i,0))
    
df_test['drug_cat'] = drug_cat

In [133]:
from gensim.models import Word2Vec

i=0
list_of_sent =[]
for sent in df_train['cleaned_text'].values:
    list_of_sent.append(sent.split())
    
w2v_model=Word2Vec(list_of_sent,min_count=5,size=300, workers=4)

w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

sent_vectors = [] #List to store all the avg w2vec's 

for sent in tqdm(df_train['cleaned_text']):
    cnt = 0 #to count no of words in each reviews
    sent_vec = np.zeros(300) #Initializing with zeroes
#     print("sent:",sent) 
#     sent = sent.decode("utf-8") 
    for word in sent.split():
        try:
#             print(word)
            wvec = w2v_model.wv[word] #Vector of each using w2v model
#             print("wvec:",wvec)
            sent_vec += wvec #Adding the vectors
#             print("sent_vec:",sent_vec)
            cnt += 1
        except: 
            pass #When the word is not in the dictionary then do nothing  
#     print(sent_vec)
    sent_vec /= cnt 
    sent_vectors.append(sent_vec) 
sent_vectors_train = np.array(sent_vectors)
# savefile(sent_vectors , 'AvgW2V_RF_Train')

number of words that occured minimum 5 times  12748
sample words  ['autoimmune', 'diseases', 'tend', 'come', 'clusters', 'gilenya', 'feel', 'good', 'not', 'think', 'change', 'anything', 'waste', 'time', 'energy', 'taking', 'tysabri', 'amazing', 'no', 'symptoms', 'dodgy', 'color', 'vision', 'since', 'always', 'know', 'last', 'month', 'year', 'decade', 'ive', 'decided', 'enjoy', 'ride', 'point', 'worrying', 'completely', 'understand', 'would', 'want', 'try', 'results', 'reported', 'stand', 'peer', 'review', 'publication', 'much', 'still', 'convincing']


100%|█████████████████████████████████████| 5279/5279 [00:07<00:00, 702.02it/s]


In [134]:
from gensim.models import Word2Vec

# i=0
# list_of_sent =[]
# for sent in reviews_test['cleaned_text'].values:
#     list_of_sent.append(sent.split())
    
# w2v_model=Word2Vec(list_of_sent,min_count=5,size=300, workers=4)

w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

sent_vectors = [] #List to store all the avg w2vec's 

for sent in tqdm(df_test['cleaned_text']):
    cnt = 0 #to count no of words in each reviews
    sent_vec = np.zeros(300) #Initializing with zeroes
#     print("sent:",sent) 
#     sent = sent.decode("utf-8") 
    for word in sent.split():
        try:
#             print(word)
            wvec = w2v_model.wv[word] #Vector of each using w2v model
#             print("wvec:",wvec)
            sent_vec += wvec #Adding the vectors
#             print("sent_vec:",sent_vec)
            cnt += 1
        except: 
            pass #When the word is not in the dictionary then do nothing  
#     print(sent_vec)
    sent_vec /= cnt 
    sent_vectors.append(sent_vec) 
sent_vectors_test = np.array(sent_vectors)
# savefile(sent_vectors , 'AvgW2V_RF_Train')

number of words that occured minimum 5 times  12748
sample words  ['autoimmune', 'diseases', 'tend', 'come', 'clusters', 'gilenya', 'feel', 'good', 'not', 'think', 'change', 'anything', 'waste', 'time', 'energy', 'taking', 'tysabri', 'amazing', 'no', 'symptoms', 'dodgy', 'color', 'vision', 'since', 'always', 'know', 'last', 'month', 'year', 'decade', 'ive', 'decided', 'enjoy', 'ride', 'point', 'worrying', 'completely', 'understand', 'would', 'want', 'try', 'results', 'reported', 'stand', 'peer', 'review', 'publication', 'much', 'still', 'convincing']


100%|█████████████████████████████████████| 2924/2924 [00:04<00:00, 595.34it/s]


In [135]:
reviews_train_bow = np.column_stack((sent_vectors_train,df_train['drug_cat'].values))
reviews_test_bow = np.column_stack((sent_vectors_test,df_test['drug_cat'].values))

In [137]:
from sklearn.metrics import f1_score
def evaluate_macroF1_lgb(truth, predictions):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [164]:
import lightgbm as lgb
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
fit_params={"early_stopping_rounds":300, 
            "eval_metric" : evaluate_macroF1_lgb, 
            "eval_set" : [(reviews_train_bow,df_train['sentiment'].values)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': True,
            'categorical_feature':[300] }
# fit_params={"early_stopping_rounds":300, 
#             "eval_metric" : evaluate_macroF1_lgb, 
#             "eval_set" : [(reviews_train_bow,df_train['sentiment'].values)],
#             'eval_names': ['valid'],
#             #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
#             'verbose': True }

param_test ={'num_leaves': sp_randint(12, 20), 
             'min_child_samples': sp_randint(40, 100), 
             #'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.75, scale=0.25), 
             'colsample_bytree': sp_uniform(loc=0.8, scale=0.15)#,
             #'reg_alpha': [0, 1e-3, 1e-1, 1, 10, 50, 100],
             #'reg_lambda': [0, 1e-3, 1e-1, 1, 10, 50, 100]
            }

In [169]:
clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.05, objective='multiclass',
                         random_state=15, silent=True, metric='None', 
                         n_jobs=4, n_estimators=5000)

In [170]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=10,
    scoring='f1_macro',
    cv=5,
    refit=True,
    random_state=15,
    verbose=True)

_ = gs.fit(reviews_train_bow,df_train['sentiment'].values, **fit_params)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Amit.Peshwani\AppData\Local\Continuum\anaconda3\envs\doc-intel\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [300]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Amit.Peshwani\AppData\Local\Continuum\anaconda3\envs\doc-intel\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[1]	valid's macroF1: 0.280097
Training until validation scores don't improve for 300 rounds.
[2]	valid's macroF1: 0.280097
[3]	valid's macroF1: 0.280097
[4]	valid's macroF1: 0.280097
[5]	valid's macroF1: 0.280097
[6]	valid's macroF1: 0.280097
[7]	valid's macroF1: 0.280097
[8]	valid's macroF1: 0.280097
[9]	valid's macroF1: 0.280097
[10]	valid's macroF1: 0.280097
[11]	valid's macroF1: 0.280097
[12]	valid's macroF1: 0.280097
[13]	valid's macroF1: 0.280097
[14]	valid's macroF1: 0.280097
[15]	valid's macroF1: 0.280097
[16]	valid's macroF1: 0.280097
[17]	valid's macroF1: 0.281747
[18]	valid's macroF1: 0.287466
[19]	valid's macroF1: 0.29144
[20]	valid's macroF1: 0.296211
[21]	valid's macroF1: 0.30318
[22]	valid's macroF1: 0.311585
[23]	valid's macroF1: 0.321089
[24]	valid's macroF1: 0.325349
[25]	valid's macroF1: 0.333579
[26]	valid's macroF1: 0.336392
[27]	valid's macroF1: 0.341124
[28]	valid's macroF1: 0.346984
[29]	valid's macroF1: 0.350081
[30]	valid's macroF1: 0.353792
[31]	valid's macro

[259]	valid's macroF1: 0.864908
[260]	valid's macroF1: 0.865618
[261]	valid's macroF1: 0.866329
[262]	valid's macroF1: 0.868562
[263]	valid's macroF1: 0.869865
[264]	valid's macroF1: 0.870678
[265]	valid's macroF1: 0.871677
[266]	valid's macroF1: 0.872268
[267]	valid's macroF1: 0.872969
[268]	valid's macroF1: 0.873668
[269]	valid's macroF1: 0.875172
[270]	valid's macroF1: 0.875172
[271]	valid's macroF1: 0.875542
[272]	valid's macroF1: 0.875836
[273]	valid's macroF1: 0.876932
[274]	valid's macroF1: 0.878103
[275]	valid's macroF1: 0.879341
[276]	valid's macroF1: 0.879948
[277]	valid's macroF1: 0.880469
[278]	valid's macroF1: 0.881051
[279]	valid's macroF1: 0.882428
[280]	valid's macroF1: 0.882428
[281]	valid's macroF1: 0.883982
[282]	valid's macroF1: 0.883462
[283]	valid's macroF1: 0.884832
[284]	valid's macroF1: 0.885803
[285]	valid's macroF1: 0.887167
[286]	valid's macroF1: 0.887644
[287]	valid's macroF1: 0.888218
[288]	valid's macroF1: 0.88915
[289]	valid's macroF1: 0.889932
[290]	val

[515]	valid's macroF1: 0.914724
[516]	valid's macroF1: 0.914724
[517]	valid's macroF1: 0.914956
[518]	valid's macroF1: 0.914956
[519]	valid's macroF1: 0.914956
[520]	valid's macroF1: 0.914956
[521]	valid's macroF1: 0.914956
[522]	valid's macroF1: 0.914956
[523]	valid's macroF1: 0.914956
[524]	valid's macroF1: 0.914956
[525]	valid's macroF1: 0.914956
[526]	valid's macroF1: 0.914956
[527]	valid's macroF1: 0.914956
[528]	valid's macroF1: 0.914808
[529]	valid's macroF1: 0.914808
[530]	valid's macroF1: 0.914808
[531]	valid's macroF1: 0.914808
[532]	valid's macroF1: 0.914808
[533]	valid's macroF1: 0.914808
[534]	valid's macroF1: 0.914808
[535]	valid's macroF1: 0.914808
[536]	valid's macroF1: 0.914808
[537]	valid's macroF1: 0.914808
[538]	valid's macroF1: 0.914808
[539]	valid's macroF1: 0.91465
[540]	valid's macroF1: 0.91465
[541]	valid's macroF1: 0.91465
[542]	valid's macroF1: 0.914269
[543]	valid's macroF1: 0.914269
[544]	valid's macroF1: 0.91465
[545]	valid's macroF1: 0.914576
[546]	valid'

...

[259]	valid's macroF1: 0.759065
[260]	valid's macroF1: 0.760834
[261]	valid's macroF1: 0.761564
[262]	valid's macroF1: 0.762138
[263]	valid's macroF1: 0.763381
[264]	valid's macroF1: 0.764622
[265]	valid's macroF1: 0.76426
[266]	valid's macroF1: 0.766734
[267]	valid's macroF1: 0.766876
[268]	valid's macroF1: 0.767004
[269]	valid's macroF1: 0.769326
[270]	valid's macroF1: 0.770753
[271]	valid's macroF1: 0.772133
[272]	valid's macroF1: 0.773904
[273]	valid's macroF1: 0.774256
[274]	valid's macroF1: 0.774572
[275]	valid's macroF1: 0.774925
[276]	valid's macroF1: 0.776438
[277]	valid's macroF1: 0.777493
[278]	valid's macroF1: 0.776282
[279]	valid's macroF1: 0.77679
[280]	valid's macroF1: 0.780257
[281]	valid's macroF1: 0.781111
[282]	valid's macroF1: 0.781316
[283]	valid's macroF1: 0.782011
[284]	valid's macroF1: 0.7839
[285]	valid's macroF1: 0.7839
[286]	valid's macroF1: 0.783867
[287]	valid's macroF1: 0.785562
[288]	valid's macroF1: 0.787322
[289]	valid's macroF1: 0.78801
[290]	valid's m

[515]	valid's macroF1: 0.968248
[516]	valid's macroF1: 0.967911
[517]	valid's macroF1: 0.968348
[518]	valid's macroF1: 0.969532
[519]	valid's macroF1: 0.969277
[520]	valid's macroF1: 0.969612
[521]	valid's macroF1: 0.969277
[522]	valid's macroF1: 0.969947
[523]	valid's macroF1: 0.971027
[524]	valid's macroF1: 0.971027
[525]	valid's macroF1: 0.971027
[526]	valid's macroF1: 0.971361
[527]	valid's macroF1: 0.971283
[528]	valid's macroF1: 0.971525
[529]	valid's macroF1: 0.972113
[530]	valid's macroF1: 0.972956
[531]	valid's macroF1: 0.974383
[532]	valid's macroF1: 0.974715
[533]	valid's macroF1: 0.975553
[534]	valid's macroF1: 0.975884
[535]	valid's macroF1: 0.975884
[536]	valid's macroF1: 0.975884
[537]	valid's macroF1: 0.97563
[538]	valid's macroF1: 0.975884
[539]	valid's macroF1: 0.976137
[540]	valid's macroF1: 0.975806
[541]	valid's macroF1: 0.97563
[542]	valid's macroF1: 0.976389
[543]	valid's macroF1: 0.976137
[544]	valid's macroF1: 0.976797
[545]	valid's macroF1: 0.976797
[546]	vali

[772]	valid's macroF1: 0.998886
[773]	valid's macroF1: 0.998886
[774]	valid's macroF1: 0.998886
[775]	valid's macroF1: 0.998886
[776]	valid's macroF1: 0.998886
[777]	valid's macroF1: 0.998886
[778]	valid's macroF1: 0.998886
[779]	valid's macroF1: 0.998886
[780]	valid's macroF1: 0.998886
[781]	valid's macroF1: 0.998886
[782]	valid's macroF1: 0.998886
[783]	valid's macroF1: 0.998886
[784]	valid's macroF1: 0.998886
[785]	valid's macroF1: 0.998886
[786]	valid's macroF1: 0.998886
[787]	valid's macroF1: 0.998886
[788]	valid's macroF1: 0.998886
[789]	valid's macroF1: 0.998886
[790]	valid's macroF1: 0.998886
[791]	valid's macroF1: 0.998886
[792]	valid's macroF1: 0.998886
[793]	valid's macroF1: 0.998886
[794]	valid's macroF1: 0.998886
[795]	valid's macroF1: 0.998886
[796]	valid's macroF1: 0.998886
[797]	valid's macroF1: 0.998886
[798]	valid's macroF1: 0.998886
[799]	valid's macroF1: 0.998886
[800]	valid's macroF1: 0.998886
[801]	valid's macroF1: 0.998886
[802]	valid's macroF1: 0.998886
[803]	va

[1027]	valid's macroF1: 0.999443
[1028]	valid's macroF1: 0.999443
[1029]	valid's macroF1: 0.999443
[1030]	valid's macroF1: 0.999443
[1031]	valid's macroF1: 0.999443
[1032]	valid's macroF1: 0.999443
[1033]	valid's macroF1: 0.999443
[1034]	valid's macroF1: 0.999443
[1035]	valid's macroF1: 0.999443
[1036]	valid's macroF1: 0.999443
[1037]	valid's macroF1: 0.999443
[1038]	valid's macroF1: 0.999443
[1039]	valid's macroF1: 0.999444
[1040]	valid's macroF1: 0.999444
[1041]	valid's macroF1: 0.999444
[1042]	valid's macroF1: 0.999444
[1043]	valid's macroF1: 0.999444
[1044]	valid's macroF1: 0.999444
[1045]	valid's macroF1: 0.999444
[1046]	valid's macroF1: 0.999444
[1047]	valid's macroF1: 0.999444
[1048]	valid's macroF1: 0.999444
[1049]	valid's macroF1: 0.999444
[1050]	valid's macroF1: 0.999444
[1051]	valid's macroF1: 0.999444
[1052]	valid's macroF1: 0.999687
[1053]	valid's macroF1: 0.999687
[1054]	valid's macroF1: 0.999687
[1055]	valid's macroF1: 0.999687
[1056]	valid's macroF1: 0.999687
[1057]	val

[1275]	valid's macroF1: 0.999687
[1276]	valid's macroF1: 0.999687
[1277]	valid's macroF1: 0.999687
[1278]	valid's macroF1: 0.999687
[1279]	valid's macroF1: 0.999687
[1280]	valid's macroF1: 0.999687
[1281]	valid's macroF1: 0.999687
[1282]	valid's macroF1: 0.999687
[1283]	valid's macroF1: 0.999687
[1284]	valid's macroF1: 0.999687
[1285]	valid's macroF1: 0.999687
[1286]	valid's macroF1: 0.999687
[1287]	valid's macroF1: 0.999687
[1288]	valid's macroF1: 0.999687
[1289]	valid's macroF1: 0.999687
[1290]	valid's macroF1: 0.999687
[1291]	valid's macroF1: 0.999687
[1292]	valid's macroF1: 0.999687
[1293]	valid's macroF1: 0.999687
[1294]	valid's macroF1: 0.999687
[1295]	valid's macroF1: 0.999687
[1296]	valid's macroF1: 0.999687
[1297]	valid's macroF1: 0.999687
[1298]	valid's macroF1: 0.999687
[1299]	valid's macroF1: 0.999687
[1300]	valid's macroF1: 0.999687
[1301]	valid's macroF1: 0.999687
[1302]	valid's macroF1: 0.999687
[1303]	valid's macroF1: 0.999687
[1304]	valid's macroF1: 0.999687
[1305]	val

In [171]:
gs.best_params_

{'colsample_bytree': 0.8559180467491833,
 'min_child_samples': 90,
 'num_leaves': 13,
 'subsample': 0.8683686106707903}

In [174]:
clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.05, objective='multiclass',
                         random_state=15, silent=True, metric='None', 
                         n_jobs=4, n_estimators=100,colsample_bytree= 0.8559180467491833,
 min_child_samples= 90,
 num_leaves= 13,
 subsample= 0.8683686106707903)

In [175]:
clf.fit(reviews_train_bow,df_train['sentiment'].values, **fit_params, categorical_feature=[300])

TypeError: fit() got multiple values for keyword argument 'categorical_feature'

In [152]:
clf.fit(reviews_train_bow,df_train['sentiment'].values, **fit_params, categorical_feature=[300])

C:\Users\Amit.Peshwani\AppData\Local\Continuum\anaconda3\envs\doc-intel\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [300]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Amit.Peshwani\AppData\Local\Continuum\anaconda3\envs\doc-intel\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[1]	valid's macroF1: 0.280097
Training until validation scores don't improve for 300 rounds.
[2]	valid's macroF1: 0.280097
[3]	valid's macroF1: 0.280097
[4]	valid's macroF1: 0.280097
[5]	valid's macroF1: 0.280097
[6]	valid's macroF1: 0.280097
[7]	valid's macroF1: 0.280097
[8]	valid's macroF1: 0.280097
[9]	valid's macroF1: 0.280097
[10]	valid's macroF1: 0.280097
[11]	valid's macroF1: 0.280097
[12]	valid's macroF1: 0.280097
[13]	valid's macroF1: 0.280923
[14]	valid's macroF1: 0.280923
[15]	valid's macroF1: 0.287466
[16]	valid's macroF1: 0.295796
[17]	valid's macroF1: 0.305533
[18]	valid's macroF1: 0.314641
[19]	valid's macroF1: 0.324382
[20]	valid's macroF1: 0.336002
[21]	valid's macroF1: 0.345561
[22]	valid's macroF1: 0.351493
[23]	valid's macroF1: 0.361388
[24]	valid's macroF1: 0.379834
[25]	valid's macroF1: 0.390448
[26]	valid's macroF1: 0.398043
[27]	valid's macroF1: 0.407922
[28]	valid's macroF1: 0.415748
[29]	valid's macroF1: 0.42501
[30]	valid's macroF1: 0.431866
[31]	valid's macr

[259]	valid's macroF1: 0.998328
[260]	valid's macroF1: 0.998328
[261]	valid's macroF1: 0.998328
[262]	valid's macroF1: 0.998328
[263]	valid's macroF1: 0.998328
[264]	valid's macroF1: 0.998328
[265]	valid's macroF1: 0.998328
[266]	valid's macroF1: 0.998328
[267]	valid's macroF1: 0.998643
[268]	valid's macroF1: 0.998643
[269]	valid's macroF1: 0.998643
[270]	valid's macroF1: 0.998643
[271]	valid's macroF1: 0.998643
[272]	valid's macroF1: 0.998643
[273]	valid's macroF1: 0.998643
[274]	valid's macroF1: 0.998957
[275]	valid's macroF1: 0.998957
[276]	valid's macroF1: 0.998957
[277]	valid's macroF1: 0.998957
[278]	valid's macroF1: 0.998957
[279]	valid's macroF1: 0.998957
[280]	valid's macroF1: 0.998957
[281]	valid's macroF1: 0.998957
[282]	valid's macroF1: 0.998957
[283]	valid's macroF1: 0.998957
[284]	valid's macroF1: 0.998957
[285]	valid's macroF1: 0.998957
[286]	valid's macroF1: 0.9992
[287]	valid's macroF1: 0.9992
[288]	valid's macroF1: 0.9992
[289]	valid's macroF1: 0.9992
[290]	valid's ma

KeyboardInterrupt: 